<a href="https://colab.research.google.com/github/Praveengovianalytics/Building-Trustworthy-LLM-Apps-and-Evaluating-LLM-agents-/blob/main/Evaluate_RAG_with_Trulens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 Evaluate RAG with Trulens

In this quickstart you will create a RAG from scratch and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/quickstart/quickstart.ipynb)

In [ ]:
! pip install trulens_eval==0.23.0 chromadb==0.4.18 openai==1.3.7 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.1/657.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [ ]:
llm_details = """
Large language model definition
A large language model (LLM) is a deep learning algorithm that can perform a variety of natural language processing (NLP) tasks. Large language models use transformer models and are trained using massive datasets — hence, large. This enables them to recognize, translate, predict, or generate text or other content.

Large language models are also referred to as neural networks (NNs), which are computing systems inspired by the human brain. These neural networks work using a network of nodes that are layered, much like neurons.

In addition to teaching human languages to artificial intelligence (AI) applications, large language models can also be trained to perform a variety of tasks like understanding protein structures, writing software code, and more. Like the human brain, large language models must be pre-trained and then fine-tuned so that they can solve text classification, question answering, document summarization, and text generation problems. Their problem-solving capabilities can be applied to fields like healthcare, finance, and entertainment where large language models serve a variety of NLP applications, such as translation, chatbots, AI assistants, and so on.

Large language models also have large numbers of parameters, which are akin to memories the model collects as it learns from training. Think of these parameters as the model’s knowledge bank.

So, what is a transformer model?
A transformer model is the most common architecture of a large language model. It consists of an encoder and a decoder. A transformer model processes data by tokenizing the input, then simultaneously conducting mathematical equations to discover relationships between tokens. This enables the computer to see the patterns a human would see were it given the same query.

Transformer models work with self-attention mechanisms, which enables the model to learn more quickly than traditional models like long short-term memory models. Self-attention is what enables the transformer model to consider different parts of the sequence, or the entire context of a sentence, to generate predictions.

Related: Apply transformers to your search applications

Key components of large language models
Large language models are composed of multiple neural network layers. Recurrent layers, feedforward layers, embedding layers, and attention layers work in tandem to process the input text and generate output content.

The embedding layer creates embeddings from the input text. This part of the large language model captures the semantic and syntactic meaning of the input, so the model can understand context.

The feedforward layer (FFN) of a large language model is made of up multiple fully connected layers that transform the input embeddings. In so doing, these layers enable the model to glean higher-level abstractions — that is, to understand the user's intent with the text input.

The recurrent layer interprets the words in the input text in sequence. It captures the relationship between words in a sentence.

The attention mechanism enables a language model to focus on single parts of the input text that is relevant to the task at hand. This layer allows the model to generate the most accurate outputs.

There are three main kinds of large language models:

Generic or raw language models predict the next word based on the language in the training data. These language models perform information retrieval tasks.
Instruction-tuned language models are trained to predict responses to the instructions given in the input. This allows them to perform sentiment analysis, or to generate text or code.
Dialog-tuned language models are trained to have a dialog by predicting the next response. Think of chatbots or conversational AI.
What is the difference between large language models and generative AI?
Generative AI is an umbrella term that refers to artificial intelligence models that have the capability to generate content. Generative AI can generate text, code, images, video, and music. Examples of generative AI include Midjourney, DALL-E, and ChatGPT.

Large language models are a type of generative AI that are trained on text and produce textual content. ChatGPT is a popular example of generative text AI.

All large language models are generative AI1.

How do large language models work?
A large language model is based on a transformer model and works by receiving an input, encoding it, and then decoding it to produce an output prediction. But before a large language model can receive text input and generate an output prediction, it requires training, so that it can fulfill general functions, and fine-tuning, which enables it to perform specific tasks.

Training: Large language models are pre-trained using large textual datasets from sites like Wikipedia, GitHub, or others. These datasets consist of trillions of words, and their quality will affect the language model's performance. At this stage, the large language model engages in unsupervised learning, meaning it processes the datasets fed to it without specific instructions. During this process, the LLM's AI algorithm can learn the meaning of words, and of the relationships between words. It also learns to distinguish words based on context. For example, it would learn to understand whether "right" means "correct," or the opposite of "left."

Fine-tuning: In order for a large language model to perform a specific task, such as translation, it must be fine-tuned to that particular activity. Fine-tuning optimizes the performance of specific tasks.

Prompt-tuning fulfills a similar function to fine-tuning, whereby it trains a model to perform a specific task through few-shot prompting, or zero-shot prompting. A prompt is an instruction given to an LLM. Few-shot prompting teaches the model to predict outputs through the use of examples. For instance, in this sentiment analysis exercise, a few-shot prompt would look like this:

Customer review: This plant is so beautiful!
Customer sentiment: positive

Customer review: This plant is so hideous!
Customer sentiment: negative
The language model would understand, through the semantic meaning of "hideous," and because an opposite example was provided, that the customer sentiment in the second example is "negative."

Alternatively, zero-shot prompting does not use examples to teach the language model how to respond to inputs. Instead, it formulates the question as "The sentiment in ‘This plant is so hideous' is…." It clearly indicates which task the language model should perform, but does not provide problem-solving examples.

Large language models use cases
Large language models can be used for several purposes:

Information retrieval: Think of Bing or Google. Whenever you use their search feature, you are relying on a large language model to produce information in response to a query. It's able to retrieve information, then summarize and communicate the answer in a conversational style.
Sentiment analysis: As applications of natural language processing, large language models enable companies to analyze the sentiment of textual data.
Text generation: Large language models are behind generative AI, like ChatGPT, and can generate text based on inputs. They can produce an example of text when prompted. For example: "Write me a poem about palm trees in the style of Emily Dickinson."
Code generation: Like text generation, code generation is an application of generative AI. LLMs understand patterns, which enables them to generate code.
Chatbots and conversational AI: Large language models enable customer service chatbots or conversational AI to engage with customers, interpret the meaning of their queries or responses, and offer responses in turn.
In addition to these use cases, large language models can complete sentences, answer questions, and summarize text.

With such a wide variety of applications, large language applications can be found in a multitude of fields:

Tech: Large language models are used anywhere from enabling search engines to respond to queries, to assisting developers with writing code.
Healthcare and Science: Large language models have the ability to understand proteins, molecules, DNA, and RNA. This position allows LLMs to assist in the development of vaccines, finding cures for illnesses, and improving preventative care medicines. LLMs are also used as medical chatbots to perform patient intakes or basic diagnoses.
Customer Service: LLMs are used across industries for customer service purposes such as chatbots or conversational AI.
Marketing: Marketing teams can use LLMs to perform sentiment analysis to quickly generate campaign ideas or text as pitching examples, and much more.
Legal: From searching through massive textual datasets to generating legalese, large language models can assist lawyers, paralegals, and legal staff.
Banking: LLMs can support credit card companies in detecting fraud.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [ ]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=llm_details
    )

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.025398574769496918, 0.021891215816140175, -0.003533829702064395, -0.010336782783269882, 0.00013731641229242086, -0.00588309857994318, -0.0031102995853871107, 0.019733859226107597, -0.019826505333185196, -0.0589236319065094, 0.013751494698226452, 0.03330006077885628, -0.022407392039895058, 0.010760312899947166, -0.0027347474824637175, 0.006481996737420559, 0.020872095599770546, 0.011428697034716606, 0.009582369588315487, -0.02010444737970829, -0.0026007399428635836, -0.015458851121366024, -0.01845003291964531, -0.028667697682976723, -0.015220615081489086, 0.006601114757359028, 0.02664269506931305, -0.04674714058637619, -0.00989339966326952, 0.006534938234835863, 0.013791200704872608, 0.01801326684653759, -0.022592686116695404, 0.004179051611572504, 0.009886782616376877, 0.007736043073236942, 0.030255936086177826, -0.005211406387388706, 0.019098563119769096, -0.02144121378660202, 0.023982394486665726, 0.0399177186191082, 0.00976104661

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="llm_details",
                                                      embedding_function=embedding_function)

Add the university_info to the embedding database.

In [ ]:
vector_store.add("llm_details", documents=llm_details)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content":
            f"We have provided context information below. \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please answer the question: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG-Ask_about_LLM',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [ ]:
with tru_rag as recording:
    rag.query("Does LLM help to write codes ?")

In [ ]:
tru.get_leaderboard(app_ids=["RAG-Ask_about_LLM"])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
RAG-Ask_about_LLM,0.766667,1.0,1.0,2.0,0.00284


In [ ]:
tru.run_dashboard()

Starting dashboard ...
npx: installed 22 in 3.86s

Go to this url and submit the ip given here. your url is: https://common-dancers-roll.loca.lt

  Submit this IP Address: 34.106.72.115



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>